# pandas怎样对每个分组应用apply函数
**知识点: pandas的groupby遵从split、 apply、 combine模式**

<img src='./image/groupby_model.png'>

这里的split值得是pandas的groupby, 我们自己实现apply函数, apply返回的结果有pandas进行combine得到结果

**`GroupBy.apply(function)`**
- function的第一参数是DataFrame
- function的返回结果, 可以是DataFrame、 Series、 单个值和输入的DataFrame完全没关系

**本次示例:**
1. 怎样对数值列先分组然后进行归一化
2. 怎样取每个分组的TOPN数据

## 实例1: 怎样对数值案列分组归一化
将不同范围的数值列进行归一化, 映射到[0, 1]区间:
- 更容易做数据横向对比, 比如价格字段是几百到几千, 增幅字段是0到100
- 机器学习模型学的更快性能更好

归一化的公式:
<img src="./image/Normalization_Formula.png">

**演示: 用户对电影评分的归一化**
每个用户的评分不同, 有的乐观评分高, 有的悲观评分低, 按用户做归一化

In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('./data/movies/ratings.csv')

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
from pandas import DataFrame


# 实现安装用户ID分组, 然后对其中一列归一化
def ratings_norm(df: DataFrame):
    min_value = df['rating'].min()
    max_value = df['rating'].max()
    df['rating_norm'] = df['rating'].apply(lambda d: (d - min_value) / (max_value - min_value))
    return df


ratings = ratings.groupby(by='userId').apply(ratings_norm)

In [5]:
ratings[ratings['userId'] == 1].head()

userId  movieId  rating  timestamp  rating_norm
userId                                                   
1      0       1        1     4.0  964982703         0.75
       1       1        3     4.0  964981247         0.75
       2       1        6     4.0  964982224         0.75
       3       1       47     5.0  964983815         1.00
       4       1       50     5.0  964982931         1.00

可以看到userId==1这个用户, rating==4.0是他的最低分, 是个乐观派, 我们归一化为0.
## 实例2. 怎样取没个分组的TOPN数据
获取2018年每个月温度最高的2天数据

In [6]:
weather_df = pd.read_excel('./data/weather/weater_beijing.xlsx')
weather_df.head()

,日期,最高温,最低温,天气,风力风向,空气质量指数
0,2011-01-01 周六,-2°,-7°,多云~阴,无持续风向微风,NaN
1,2011-01-02 周日,-2°,-7°,多云,无持续风向微风,NaN
2,2011-01-03 周一,-2°,-6°,多云~阴,西北风~北风3-4级~4-5级,NaN
3,2011-01-04 周二,-2°,-9°,晴,北风5-6级,NaN
4,2011-01-05 周三,-2°,-10°,晴,北风~无持续风向3-4级~微风,NaN


In [7]:
weather_df['最高温'].fillna('0', inplace=True)
weather_df.loc[:, '最高温'] = weather_df['最高温'].str.replace('°', '').replace('', '0')
weather_df.loc[:, '最高温'] = weather_df['最高温'].astype('int32')

In [8]:
weather_df['最低温'].fillna('0', inplace=True)
weather_df.loc[:, '最低温'] = weather_df['最低温'].str.replace('°', '').replace('', '0')
weather_df.loc[:, '最低温'] = weather_df['最低温'].astype('int32')

In [9]:
weather_df.head()

,日期,最高温,最低温,天气,风力风向,空气质量指数
0,2011-01-01 周六,-2,-7,多云~阴,无持续风向微风,NaN
1,2011-01-02 周日,-2,-7,多云,无持续风向微风,NaN
2,2011-01-03 周一,-2,-6,多云~阴,西北风~北风3-4级~4-5级,NaN
3,2011-01-04 周二,-2,-9,晴,北风5-6级,NaN
4,2011-01-05 周三,-2,-10,晴,北风~无持续风向3-4级~微风,NaN


In [10]:
# 新增一列为月份
weather_df['月份'] = weather_df['日期'].str[:7]
weather_df.head()

,日期,最高温,最低温,天气,风力风向,空气质量指数,月份
0,2011-01-01 周六,-2,-7,多云~阴,无持续风向微风,NaN,2011-01
1,2011-01-02 周日,-2,-7,多云,无持续风向微风,NaN,2011-01
2,2011-01-03 周一,-2,-6,多云~阴,西北风~北风3-4级~4-5级,NaN,2011-01
3,2011-01-04 周二,-2,-9,晴,北风5-6级,NaN,2011-01
4,2011-01-05 周三,-2,-10,晴,北风~无持续风向3-4级~微风,NaN,2011-01


In [11]:
def get_wendu_topn(df: DataFrame, topn: int):
    """
    获取温度最高的topn

    Args:
        df (): 每个月份分组group的DataFrame
        topn ():

    Returns:

    """
    # 把数据更具最高温
    sort_data = df.sort_values(by='最高温')
    return sort_data[['日期', '最高温']][-topn:]


result = weather_df.groupby('月份').apply(get_wendu_topn, topn=2)

In [12]:
result.head()

日期 最高温
月份                           
2011-01 18  2011-01-30 周日   3
        19  2011-01-31 周一   7
2011-02 38  2011-02-19 周六  10
        43  2011-02-24 周四  11
2011-03 77  2011-03-30 周三  22

In [13]:
for i in result:
    print(i)

日期
最高温


In [34]:
r = result.loc['2011-01']

In [37]:
r.loc[r.index==18]

,日期,最高温
18,2011-01-30 周日,3
